In [0]:
import pyspark.sql.functions as F

In [0]:
containerName = "container_name"
storageAccountName = "storage_account_name"
sas = "sas_token"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

In [0]:
dbutils.fs.mount(
  source = "wasbs://container_name@storage_account_name.blob.core.windows.net/linkedin_jobs.csv",
  mount_point = "/mnt/linkedin_jobs_data_data",
  extra_configs = {config:sas})

Out[12]: True

In [0]:
spark.read \
.option("header","true") \
.option("inferSchema", "true") \
.csv("/mnt/linkedin_jobs_data_data") \
.createOrReplaceTempView('linkedin_jobs')

In [0]:
df = spark.table('linkedin_jobs')

df.limit(6).toPandas()

,company_name,level,job_type,department,position,location,date_posted,link_job
0,Luxoft,Mid-Senior level,Full-time,Not specified,Scrum Master with Spanish,Poland,3 weeks ago,https://www.linkedin.com/jobs/view/3344772217/...
1,Danfoss,Mid-Senior level,Full-time,Mechanical Or Industrial Engineering,Commerce Scrum Master,Chwaszczyno,1 day ago,https://www.linkedin.com/jobs/view/3340671774/...
2,Sii Poland,Mid-Senior level,Full-time,Information Technology & Services,Scrum Master,Gdańsk,1 week ago,https://www.linkedin.com/jobs/view/3327168022/...
3,EY,Mid-Senior level,Full-time,Accounting,SAP PP/PM/QM Consultant,Poznań,1 day ago,https://www.linkedin.com/jobs/view/3245359120/...
4,Capgemini,Mid-Senior level,Full-time,Information Technology & Services,Team/Project Manager (BI/Big Data),Cracow,2 weeks ago,https://www.linkedin.com/jobs/view/3342989394/...
5,Majorel,Not specified,Full-time,Not specified,Data Project Manager,Poland,3 days ago,https://www.linkedin.com/jobs/view/3365659883/...


In [0]:
df =df \
.select(F.col('company_name'),
        F.col('level'),
        F.col('job_type'),
        F.col('department'),
        F.col('position'),
        F.col('location'),
        F.col('date_posted'))

In [0]:
df.limit(6)\
.toPandas()

,company_name,level,job_type,department,position,location,date_posted
0,Luxoft,Mid-Senior level,Full-time,Not specified,Scrum Master with Spanish,Poland,3 weeks ago
1,Danfoss,Mid-Senior level,Full-time,Mechanical Or Industrial Engineering,Commerce Scrum Master,Chwaszczyno,1 day ago
2,Sii Poland,Mid-Senior level,Full-time,Information Technology & Services,Scrum Master,Gdańsk,1 week ago
3,EY,Mid-Senior level,Full-time,Accounting,SAP PP/PM/QM Consultant,Poznań,1 day ago
4,Capgemini,Mid-Senior level,Full-time,Information Technology & Services,Team/Project Manager (BI/Big Data),Cracow,2 weeks ago
5,Majorel,Not specified,Full-time,Not specified,Data Project Manager,Poland,3 days ago


In [0]:
cnt_vac_by_location = df \
.where(F.col('location') != 'Poland')\
.groupBy(F.col('location'))\
.agg(
    F.count(F.col('company_name')).alias('Count_Jobs_by_Location'))\
.orderBy('Count_Jobs_by_Location', ascending = False)\
.limit(5)

display(cnt_vac_by_location)

location,Count_Jobs_by_Location
Warsaw,264
Cracow,93
Wrocław,69
Gdańsk,38
Katowice,35


Output can only be rendered in Databricks

In [0]:
cnt_vac_by_dep = df \
    .groupBy(F.col('department'))\
    .agg(
        F.count(F.col('department')).alias('Count_Jobs_by_Department'))\
    .orderBy(F.col('Count_Jobs_by_Department'), ascending = False)\
    .limit(5)


display(cnt_vac_by_dep)

department,Count_Jobs_by_Department
Not specified,264
Information Technology & Services,232
Computer Software,72
Staffing & Recruiting,59
Accounting,36


Output can only be rendered in Databricks

In [0]:
cnt_vac_by_lvl = df \
.groupBy(F.col('level'))\
.agg(
    F.count(F.col('company_name')).alias('Count_Jobs_by_Experience_Level'))\
.orderBy('Count_Jobs_by_Experience_Level', ascending = False)\
.limit(3)

display(cnt_vac_by_lvl)

level,Count_Jobs_by_Experience_Level
Mid-Senior level,537
Entry level,219
Associate,151


Output can only be rendered in Databricks

In [0]:
cnt_vac_by_company_name = df \
.groupBy(F.col('company_name'))\
.agg(
    F.count(F.col('company_name')).alias('Count_Jobs_by_Company_Name'))\
.orderBy('Count_Jobs_by_Company_Name', ascending = False)\
.limit(5)

display(cnt_vac_by_company_name)

company_name,Count_Jobs_by_Company_Name
Luxoft,66
EY,36
PwC Polska,29
Sii Poland,27
ASTEK Polska,23


Output can only be rendered in Databricks